In [5]:
import pandas as pd  # import package we need
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
df = pd.read_csv('data.csv') 
#%config InlineBackend.figure_format = 'retina' # high digit graph

In [2]:
df2 = df[['bmi', 'income']].dropna()
bmi = df2.bmi
bmi_rich = df2[df2.income == 8].bmi  # BMI of the rich
bmi_ord = df2[df2.income != 8].bmi # BMI of the ordinary

In [3]:
mu = np.mean(bmi)  # mean of all
mu

28.18812531332513

In [7]:
t_stats, p_value = stats.ttest_1samp(bmi_rich, mu)# H0 we suppose that the mean of rich is as same as the mean of all
print("p value is %.10f" % (p_value/2)) 

p value is 0.0000000000


In [8]:
# while P = 0, we can sure that it is different!

In [10]:
t_stats, p_value = stats.ttest_1samp(bmi_rich[:500], mu)  # pick only 500 data
print("p value is %.10f" % (p_value/2))

p value is 0.2906513576


In [11]:
t_stats, p_value = stats.ttest_1samp(bmi_rich[:4000], mu)  # 4000
print("p value is %.10f" % (p_value/2))

p value is 0.0000660545


In [12]:
#it shows that the more data you have and you could ensure the outcome

In [13]:
#Bootsrap!

In [14]:
def bootstrap_replicate_1d(data, func):   # sampling and get the statistics answers
    return func(np.random.choice(data, size=len(data)))  

def draw_bs_reps(data, func, size=1):
    bs_replicates = np.empty(size)  # multiple sampling
    for i in range(size):   # 
        bs_replicates[i] = bootstrap_replicate_1d(data, func)  
    return bs_replicates  # 


def bootstrap_pvalue_1samp(data, pop_stats, func, size=1): #calculating the P value with shifted data.
    sample_stats = func(data)  #    
    shift_data = data - sample_stats + pop_stats  # computing the mean values
    bs_replicates = draw_bs_reps(shift_data, func, size) # multiple values
    p = np.sum( bs_replicates < sample_stats) / size # getting the P-value
    return p

In [15]:
#data = the rich's BMI, pop_stats = all people' bmi, function = np.mean, try 10000 times
bootstrap_pvalue_1samp(bmi_rich, mu, np.mean, size=10000)

0.0

In [16]:
#data = the first 500 rich's BMI, pop_stats = all people' bmi, function = np.mean, try 10000 times
bootstrap_pvalue_1samp(bmi_rich[:500], mu, np.mean, size=10000)

0.7128

In [17]:
#data = the first 4000 rich's BMI, pop_stats = all people' bmi, function = np.mean, try 10000 times
bootstrap_pvalue_1samp(bmi_rich[:4000], mu, np.mean, size=10000)

0.0002

In [18]:
# When it comes to two independent data:rich and ordin
t_stats, p_value = stats.ttest_ind(bmi_rich, bmi_ord) # using rich and ordin as the arguements instead of rich and all
print("p value is %.10f" % (p_value/2))

p value is 0.0000000000


In [19]:
# When it comes to two independent data:rich and ordin but less data, like 500.
t_stats, p_value = stats.ttest_ind(bmi_rich[:500], bmi_ord[:500]) # using rich and ordin as the arguements instead of rich and all
print("p value is %.10f" % (p_value/2))
#the P increase and we could refuse the H0

p value is 0.1219871502


In [20]:
# When it comes to two independent data:rich and ordin a little more than 500, like 1000
t_stats, p_value = stats.ttest_ind(bmi_rich[:1000], bmi_ord[:1000]) # using rich and ordin as the arguements instead of rich and all
print("p value is %.10f" % (p_value/2))
#for this P value, we can sure that H0 is wrong.

p value is 0.0000846839


In [28]:
def diff_of_means(data_1, data_2):  
    diff = np.mean(data_2) - np.mean(data_1)  # computing the mean for two sets: rich and ordin
    return diff


def permutation_sample(data1, data2):  # generrating the new data set
    data = np.concatenate((data1, data2))  # concatenating them
    permuted_data = np.random.permutation(data)  # reindex
    perm_sample_1 = permuted_data[:len(data1)]  # devide into two parts
    perm_sample_2 = permuted_data[len(data1):]
    return perm_sample_1, perm_sample_2


def draw_perm_reps(data_1, data_2, func, size=1):  # multiple try suffle
    perm_replicates = np.empty(size)
    for i in range(size):
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    return perm_replicates


def permutation_pvalue(data_1, data_2, func_diff, size=1):  # calculating the P by sampling difference of mean 
    empirical_test_stats = func_diff(data_1, data_2)
    perm_replicates = draw_perm_reps(data_1, data_2, func_diff, size)
    p = np.sum(perm_replicates > empirical_test_stats) / len(perm_replicates)
    return p


In [22]:
#data1 = the firstrich's BMI, data2 = the BMI of ordin people, function = difference between their means, try 10000 times
permutation_pvalue(bmi_rich, bmi_ord, diff_of_means, size=10000)

0.0

In [23]:
# When it comes to two independent data:rich and ordin but less data, like 500.
permutation_pvalue(bmi_rich[:500], bmi_ord[:500], diff_of_means, size=10000)

0.1221

In [27]:
# When it comes to two independent data:rich and ordin but less data, like 1000.
permutation_pvalue(bmi_rich[:1000], bmi_ord[:1000], diff_of_means, size=10000)

0.0001

In [25]:
#with more data, we can sure that rich tends to be thin